In [ ]:
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 119.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: idna
    Found existing installation: idna 3.11
    Uninstalling idna-3.11:
      Successfully uninstalled idna-3.11


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("restifier").project("resistor-value-detector")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Resistor-Value-Detector-2 in yolov8:: 100%|██████████| 13232/13232 [00:01<00:00, 11434.31it/s]


In [ ]:
from roboflow import Roboflow
import yaml, os
from collections import defaultdict

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("restifier").project("resistor-value-detector")
version = project.version(2)
dataset = version.download("yolov8")

# Load class names
with open(os.path.join(dataset.location, "data.yaml")) as f:
    names = yaml.safe_load(f)["names"]

print("Class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

# -------------------------------
# COUNT TRAIN IMAGES PER CLASS
# -------------------------------
image_counts = defaultdict(int)   # number of images containing class
object_counts = defaultdict(int)  # total objects per class

train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if not file.endswith(".txt"):
        continue

    path = os.path.join(train_labels, file)
    classes_in_image = set()

    with open(path, "r") as f:
        for line in f:
            cls = int(line.split()[0])
            classes_in_image.add(cls)
            object_counts[cls] += 1

    for cls in classes_in_image:
        image_counts[cls] += 1

# -------------------------------
# PRINT RESULTS
# -------------------------------
print("\nTrain-only IMAGE counts:")
for i, name in enumerate(names):
    print(f"{i} ({name}): {image_counts[i]} images")

print("\nTrain-only OBJECT counts:")
for i, name in enumerate(names):
    print(f"{i} ({name}): {object_counts[i]} objects")

loading Roboflow workspace...
loading Roboflow project...
Class mapping:
0 → POT
1 → RES

Train-only IMAGE counts:
0 (POT): 36 images
1 (RES): 900 images

Train-only OBJECT counts:
0 (POT): 40 objects
1 (RES): 3287 objects


In [ ]:
from roboflow import Roboflow

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("jiaqiye_77-163-com").project("electronic-components-lzinq")
version = project.version(3)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
from collections import defaultdict
import os

def delete_resistor_from_split(split):
    lbl_path = os.path.join(dataset.location, split, "labels")
    img_path = os.path.join(dataset.location, split, "images")

    if not os.path.isdir(lbl_path):
        print(f"{split.upper()}: no '{lbl_path}' folder, skipping")
        return

    deleted = 0

    for file in os.listdir(lbl_path):
        if not file.endswith(".txt"):
            continue

        label_file = os.path.join(lbl_path, file)

        # Check if label file contains resistor annotations
        delete_this = False
        with open(label_file) as f:
            for line in f:
                cls = int(line.split()[0])
                if cls == resistor_idx:
                    delete_this = True
                    break

        if delete_this:
            # Delete label
            os.remove(label_file)

            # Delete corresponding image (.jpg or .jpeg or .png)
            base = file[:-4]
            for ext in [".jpg", ".jpeg", ".png"]:
                img_file = os.path.join(img_path, base + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)

            deleted += 1

    print(f"{split.upper()}: deleted {deleted} resistor images")


def count_split(split):
    lbl_path = os.path.join(dataset.location, split, "labels")
    counts = defaultdict(int)

    if not os.path.isdir(lbl_path):
        print(f"{split.upper()}: no '{lbl_path}' folder, skipping")
        return counts

    for file in os.listdir(lbl_path):
        if file.endswith(".txt"):
            with open(os.path.join(lbl_path, file)) as f:
                for line in f:
                    cls = int(line.split()[0])
                    counts[cls] += 1
    return counts

# -----------------------------
# Delete resistor data everywhere
# -----------------------------
for split in ["train", "valid", "test"]:
    delete_resistor_from_split(split)

# -----------------------------
# Recount class frequencies
# -----------------------------
print("\nRecounting class frequencies AFTER deletion...\n")
for split in ["train", "valid", "test"]:
    print(f"--- {split.upper()} ---")
    counts = count_split(split)
    for i, n in enumerate(names):
        print(f"{i} ({n}): {counts[i]} images")
    print()

TRAIN: deleted 0 resistor images
VALID: deleted 0 resistor images
TEST: deleted 0 resistor images

Recounting class frequencies AFTER deletion...

--- TRAIN ---
0 (POT): 40 images
1 (RES): 3287 images

--- VALID ---
0 (POT): 14 images
1 (RES): 933 images

--- TEST ---
0 (POT): 0 images
1 (RES): 188 images



In [ ]:
from collections import defaultdict
import os

def delete_resistor_from_split(split):
    lbl_path = os.path.join(dataset.location, split, "labels")
    img_path = os.path.join(dataset.location, split, "images")

    if not os.path.isdir(lbl_path):
        print(f"{split.upper()}: no '{lbl_path}' folder, skipping")
        return

    deleted = 0

    for file in os.listdir(lbl_path):
        if not file.endswith(".txt"):
            continue

        label_file = os.path.join(lbl_path, file)

        # Check if label file contains resistor annotations
        delete_this = False
        with open(label_file) as f:
            for line in f:
                cls = int(line.split()[0])
                if cls == resistor_idx:
                    delete_this = True
                    break

        if delete_this:
            # Delete label
            os.remove(label_file)

            # Delete corresponding image (.jpg or .jpeg or .png)
            base = file[:-4]
            for ext in [".jpg", ".jpeg", ".png"]:
                img_file = os.path.join(img_path, base + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)

            deleted += 1

    print(f"{split.upper()}: deleted {deleted} resistor images")


def count_split(split):
    lbl_path = os.path.join(dataset.location, split, "labels")
    counts = defaultdict(int)

    if not os.path.isdir(lbl_path):
        print(f"{split.upper()}: no '{lbl_path}' folder, skipping")
        return counts

    for file in os.listdir(lbl_path):
        if file.endswith(".txt"):
            with open(os.path.join(lbl_path, file)) as f:
                for line in f:
                    cls = int(line.split()[0])
                    counts[cls] += 1
    return counts

# --------------------------------------------------------
# Delete resistor data *ONLY* from TRAIN
# --------------------------------------------------------
delete_resistor_from_split("train")

# --------------------------------------------------------
# Recount for TRAIN only
# --------------------------------------------------------
print("\nRecounting class frequencies AFTER deletion...\n")
print("--- TRAIN ---")
counts = count_split("train")
for i, n in enumerate(names):
    print(f"{i} ({n}): {counts[i]} images")


TRAIN: deleted 0 resistor images

Recounting class frequencies AFTER deletion...

--- TRAIN ---
0 (POT): 40 images
1 (RES): 3287 images


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("new-workspace-izjtp").project("electric-components")
version = project.version(1)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electric-Components-1 in yolov8:: 100%|██████████| 614/614 [00:00<00:00, 6286.30it/s]


In [ ]:
from roboflow import Roboflow
import yaml, os
from collections import defaultdict

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("new-workspace-izjtp").project("electric-components")
version = project.version(1)
dataset = version.download("yolov8")

# -------------------------------------
# Load class names
# -------------------------------------
with open(os.path.join(dataset.location, "data.yaml")) as f:
    names = yaml.safe_load(f)["names"]

print("Class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

# resistor class index
if "resistor" not in names:
    raise ValueError("Class 'resistor' not found in the dataset.")
resistor_idx = names.index("resistor")
print(f"\nResistor class index: {resistor_idx}")

# -------------------------------------
# Function to delete all resistor images in a split
# -------------------------------------
def delete_resistor_from_split(split):
    lbl_path = os.path.join(dataset.location, split, "labels")
    img_path = os.path.join(dataset.location, split, "images")

    if not os.path.isdir(lbl_path):
        print(f"{split.upper()}: No label folder found. Skipping.")
        return

    deleted = 0

    for file in os.listdir(lbl_path):
        if not file.endswith(".txt"):
            continue

        label_file = os.path.join(lbl_path, file)

        # Detect resistor inside label file
        contains_resistor = False
        with open(label_file) as f:
            for line in f:
                cls = int(line.split()[0])
                if cls == resistor_idx:
                    contains_resistor = True
                    break

        if contains_resistor:
            # Delete label file
            os.remove(label_file)

            # Delete corresponding image (jpg/png/jpeg)
            base = file[:-4]
            for ext in [".jpg", ".jpeg", ".png"]:
                img_file = os.path.join(img_path, base + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)

            deleted += 1

    print(f"{split.upper()}: Deleted {deleted} resistor images")

# -------------------------------------
# Delete from ALL splits: train, valid, test
# -------------------------------------
for split in ["train", "valid", "test"]:
    delete_resistor_from_split(split)

# -------------------------------------
# Recount TRAIN images afterward
# -------------------------------------
print("\nTrain-only image counts AFTER deleting resistor:")
counts = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts[cls] += 1

for i, name in enumerate(names):
    print(f"{i} ({name}): {counts[i]} images")

loading Roboflow workspace...
loading Roboflow project...
Class mapping:
0 → ceramic_capacitor
1 → potentiometer
2 → resistor

Resistor class index: 2
TRAIN: Deleted 75 resistor images
VALID: Deleted 13 resistor images
TEST: Deleted 12 resistor images

Train-only image counts AFTER deleting resistor:
0 (ceramic_capacitor): 64 images
1 (potentiometer): 71 images
2 (resistor): 0 images


In [ ]:

from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("cv2024-4skvf").project("electronic-components-detection-jzf42")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electronic-components-detection-4 in yolov8:: 100%|██████████| 1268/1268 [00:00<00:00, 7640.00it/s]


In [ ]:
from roboflow import Roboflow
import yaml, os
from collections import defaultdict

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("cv2024-4skvf").project("electronic-components-detection-jzf42")
version = project.version(4)
dataset = version.download("yolov8")

# ------------------------------------------
# 1) Load class names from data.yaml
# ------------------------------------------
yaml_path = os.path.join(dataset.location, "data.yaml")

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]

print("Original class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

# ------------------------------------------
# 2) Original train-only image counts (before cleanup)
# ------------------------------------------
counts_before = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts_before[cls] += 1

print("\nTrain-only image counts (BEFORE cleanup):")
for i, name in enumerate(names):
    print(f"{i} ({name}): {counts_before[i]} images")

# ------------------------------------------
# 3) Classes to remove COMPLETELY (labels+images+YAML)
# ------------------------------------------
classes_to_remove = {
    "Resistor",
    "bornera",
    "button",
    "cristal",
    "power-resistor",
    "preset",
    "to-220",
    "to-92",
    "trafo",
}

print("\nWill remove these classes everywhere:", classes_to_remove)

# map name -> old index
name_to_old_idx = {n: i for i, n in enumerate(names)}

# Build new class list (keep only those NOT in classes_to_remove)
new_names = [n for n in names if n not in classes_to_remove]

# mapping old index -> new index (for kept classes)
old_to_new = {i: new_names.index(n) for i, n in enumerate(names) if n in new_names}

print("\nNew class mapping (after removal):")
for i, name in enumerate(new_names):
    print(f"{i} → {name}")

# ------------------------------------------
# 4) Process splits: filter labels, delete images if only removed classes
# ------------------------------------------
def process_split(split):
    label_dir = os.path.join(dataset.location, split, "labels")
    img_dir = os.path.join(dataset.location, split, "images")

    if not os.path.isdir(label_dir):
        print(f"{split.upper()}: no label folder, skipping")
        return

    removed_images = 0
    modified_labels = 0

    for lbl_file in os.listdir(label_dir):
        if not lbl_file.endswith(".txt"):
            continue

        lbl_path = os.path.join(label_dir, lbl_file)
        with open(lbl_path, "r") as f:
            lines = f.readlines()

        new_lines = []

        for line in lines:
            parts = line.strip().split()
            if not parts:
                continue

            old_cls = int(parts[0])
            class_name = names[old_cls]

            # If this object's class is in the remove list, drop it
            if class_name in classes_to_remove:
                continue

            # Keep object: map to new index
            new_cls = old_to_new[old_cls]
            parts[0] = str(new_cls)
            new_lines.append(" ".join(parts) + "\n")

        if new_lines:
            # Keep label file, but overwrite if changed
            if len(new_lines) != len(lines):
                modified_labels += 1
            with open(lbl_path, "w") as f:
                f.writelines(new_lines)
        else:
            # No remaining objects we care about → delete label + image
            os.remove(lbl_path)
            base = os.path.splitext(lbl_file)[0]
            for ext in [".jpg", ".jpeg", ".png"]:
                img_path = os.path.join(img_dir, base + ext)
                if os.path.exists(img_path):
                    os.remove(img_path)
            removed_images += 1

    print(
        f"{split.upper()}: modified {modified_labels} label files, "
        f"removed {removed_images} images containing only removed classes"
    )

# Apply to train/valid/test
for split in ["train", "valid", "test"]:
    process_split(split)

# ------------------------------------------
# 5) Update data.yaml with new names + nc
# ------------------------------------------
data["names"] = new_names
data["nc"] = len(new_names)

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f)

print("\nUpdated data.yaml written.")
print("Final class names:", new_names)

# ------------------------------------------
# 6) Recount train-only images AFTER cleanup
# ------------------------------------------
counts_after = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts_after[cls] += 1

print("\nTrain-only image counts (AFTER cleanup):")
for i, name in enumerate(new_names):
    print(f"{i} ({name}): {counts_after[i]} images")

loading Roboflow workspace...
loading Roboflow project...
Original class mapping:
0 → Resistor
1 → bornera
2 → button
3 → capacitor-ceramico
4 → capacitor-electrolitico
5 → capacitor-poliester
6 → cristal
7 → diodo
8 → ic
9 → led
10 → power-resistor
11 → preset
12 → to-220
13 → to-92
14 → trafo

Train-only image counts (BEFORE cleanup):
0 (Resistor): 2604 images
1 (bornera): 21 images
2 (button): 60 images
3 (capacitor-ceramico): 447 images
4 (capacitor-electrolitico): 722 images
5 (capacitor-poliester): 42 images
6 (cristal): 8 images
7 (diodo): 232 images
8 (ic): 202 images
9 (led): 208 images
10 (power-resistor): 0 images
11 (preset): 39 images
12 (to-220): 95 images
13 (to-92): 267 images
14 (trafo): 21 images

Will remove these classes everywhere: {'to-220', 'preset', 'Resistor', 'button', 'to-92', 'trafo', 'cristal', 'power-resistor', 'bornera'}

New class mapping (after removal):
0 → capacitor-ceramico
1 → capacitor-electrolitico
2 → capacitor-poliester
3 → diodo
4 → ic
5 → led


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("components-uncfj").project("electronics-component-dtuwz")
version = project.version(7)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electronics-Component-7 in yolov8:: 100%|██████████| 268/268 [00:00<00:00, 10190.41it/s]


In [ ]:
from roboflow import Roboflow
import yaml, os
from collections import defaultdict

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("components-uncfj").project("electronics-component-dtuwz")
version = project.version(7)
dataset = version.download("yolov8")

# --------------------------------------------------------
# Load class names
# --------------------------------------------------------
yaml_path = os.path.join(dataset.location, "data.yaml")

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]
print("\nOriginal class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

# --------------------------------------------------------
# Identify resistor class index
# --------------------------------------------------------
if "resistor" not in names:
    raise ValueError("Class 'resistor' not found in dataset names!")

resistor_idx = names.index("resistor")
print(f"\nResistor index = {resistor_idx}")

# --------------------------------------------------------
# Build new names list (remove resistor)
# --------------------------------------------------------
new_names = [n for n in names if n != "resistor"]

# mapping old index -> new index
old_to_new = {i: new_names.index(name) for i, name in enumerate(names) if name != "resistor"}

print("\nNew class mapping (after removing resistor):")
for i, name in enumerate(new_names):
    print(f"{i} → {name}")

# --------------------------------------------------------
# Function: clean each dataset split
# --------------------------------------------------------
def remove_resistor_from_split(split):
    label_dir = os.path.join(dataset.location, split, "labels")
    img_dir = os.path.join(dataset.location, split, "images")

    if not os.path.isdir(label_dir):
        print(f"{split.upper()}: no labels folder, skipping")
        return

    removed_images = 0
    modified_labels = 0

    for lbl_file in os.listdir(label_dir):
        if not lbl_file.endswith(".txt"):
            continue

        lbl_path = os.path.join(label_dir, lbl_file)
        with open(lbl_path, "r") as f:
            lines = f.readlines()

        new_lines = []

        for line in lines:
            parts = line.strip().split()
            cls = int(parts[0])

            # Skip resistor objects entirely
            if cls == resistor_idx:
                continue

            # Keep object: map class index
            new_cls = old_to_new[cls]
            parts[0] = str(new_cls)
            new_lines.append(" ".join(parts) + "\n")

        if new_lines:
            # Keep label file, write only non-resistor annotations
            if len(new_lines) != len(lines):
                modified_labels += 1
            with open(lbl_path, "w") as f:
                f.writelines(new_lines)
        else:
            # If no objects remain, delete entire image + label
            os.remove(lbl_path)
            base = lbl_file.replace(".txt", "")
            for ext in [".jpg", ".jpeg", ".png"]:
                img_file = os.path.join(img_dir, base + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)
            removed_images += 1

    print(f"{split.upper()}: modified {modified_labels} labels, removed {removed_images} images")

# Apply to train, valid, test
for split in ["train", "valid", "test"]:
    remove_resistor_from_split(split)

# --------------------------------------------------------
# Update YAML
# --------------------------------------------------------
data["names"] = new_names
data["nc"] = len(new_names)

with open(yaml_path, "w") as f:
    yaml.safe_dump(data, f)

print("\nUpdated YAML saved successfully.")

# --------------------------------------------------------
# Recount train-only images AFTER cleanup
# --------------------------------------------------------
counts_after = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts_after[cls] += 1

print("\nTrain-only image counts AFTER removing resistor:")
for i, name in enumerate(new_names):
    print(f"{i} ({name}): {counts_after[i]} images")

loading Roboflow workspace...
loading Roboflow project...

Original class mapping:
0 → Diode
1 → LED
2 → Transistor
3 → ceramic capacitors
4 → electrolytic capacitor
5 → resistor

Resistor index = 5

New class mapping (after removing resistor):
0 → Diode
1 → LED
2 → Transistor
3 → ceramic capacitors
4 → electrolytic capacitor
TRAIN: modified 0 labels, removed 13 images
VALID: modified 0 labels, removed 4 images
TEST: modified 0 labels, removed 2 images

Updated YAML saved successfully.

Train-only image counts AFTER removing resistor:
0 (Diode): 14 images
1 (LED): 20 images
2 (Transistor): 12 images
3 (ceramic capacitors): 18 images
4 (electrolytic capacitor): 13 images


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("entrenamiento-capacitor").project("electronic-components-recognition-golue")
version = project.version(1)
dataset = version.download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electronic-components-recognition-1 in yolov8:: 100%|██████████| 1718/1718 [00:00<00:00, 9462.01it/s]


In [ ]:
from roboflow import Roboflow
import yaml, os
from collections import defaultdict

rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("entrenamiento-capacitor").project("electronic-components-recognition-golue")
version = project.version(1)
dataset = version.download("yolov8")

# Load class names
with open(os.path.join(dataset.location, "data.yaml")) as f:
    names = yaml.safe_load(f)["names"]

print("Class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

# Count train-only images
counts = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts[cls] += 1

print("\nTrain-only image counts:")
for i, name in enumerate(names):
    print(f"{i} ({name}): {counts[i]} images")


loading Roboflow workspace...
loading Roboflow project...
Class mapping:
0 → 0
1 → 1
2 → 2
3 → 3
4 → 4
5 → 5
6 → 6

Train-only image counts:
0 (0): 669 images
1 (1): 804 images
2 (2): 66 images
3 (3): 65 images
4 (4): 56 images
5 (5): 68 images
6 (6): 60 images


In [ ]:
import os, yaml
from collections import defaultdict

# We already have: root, yaml_path from your previous cell
# root = dataset.location
# yaml_path = os.path.join(root, "data.yaml")

# --- Reload YAML ---
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]
print("Current class names:", names)

# find LED index (case-insensitive)
led_idx = None
for i, n in enumerate(names):
    if n.lower() == "led":
        led_idx = i
        break

if led_idx is None:
    print("No 'led' class found, nothing to delete.")
else:
    print("LED index:", led_idx)

    # Build new name list (without LED)
    new_names = [n for n in names if n.lower() != "led"]

    # Map old → new index for kept classes
    old_to_new = {
        i: new_names.index(name)
        for i, name in enumerate(names)
        if i != led_idx
    }

    print("New names (after removing LED):", new_names)
    print("Old→New id map:", old_to_new)

    def clean_split(split):
        label_dir = os.path.join(root, split, "labels")
        img_dir = os.path.join(root, split, "images")

        if not os.path.isdir(label_dir):
            print(f"{split}: no labels folder, skipping")
            return

        removed_images = 0
        modified_labels = 0

        for fname in os.listdir(label_dir):
            if not fname.endswith(".txt"):
                continue

            path = os.path.join(label_dir, fname)
            with open(path, "r") as f:
                lines = f.readlines()

            new_lines = []
            for line in lines:
                parts = line.strip().split()
                if not parts:
                    continue
                old_id = int(parts[0])

                # drop LED objects
                if old_id == led_idx:
                    continue

                # keep and remap other classes
                new_id = old_to_new[old_id]
                parts[0] = str(new_id)
                new_lines.append(" ".join(parts) + "\n")

            if new_lines:
                if len(new_lines) != len(lines):
                    modified_labels += 1
                with open(path, "w") as f:
                    f.writelines(new_lines)
            else:
                # no objects left → delete label + image
                os.remove(path)
                base = os.path.splitext(fname)[0]
                for ext in [".jpg", ".jpeg", ".png"]:
                    img_path = os.path.join(img_dir, base + ext)
                    if os.path.exists(img_path):
                        os.remove(img_path)
                removed_images += 1

        print(f"{split}: modified {modified_labels} labels, removed {removed_images} images")

    # Run on all splits
    for split in ["train", "valid", "test"]:
        clean_split(split)

    # --- Update YAML ---
    data["names"] = new_names
    data["nc"] = len(new_names)

    with open(yaml_path, "w") as f:
        yaml.safe_dump(data, f)

    print("✔ data.yaml updated. Final class names:", new_names)

Current class names: ['Diode', 'Transistor', 'ceramic capacitors', 'electrolytic capacitor']
No 'led' class found, nothing to delete.


In [ ]:
# Re-check mapping + counts
with open(yaml_path, "r") as f:
    names = yaml.safe_load(f)["names"]

print("Class mapping:")
for i, n in enumerate(names):
    print(f"{i} → {n}")

counts = defaultdict(int)
train_labels = os.path.join(root, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts[cls] += 1

print("\nTrain-only image counts:")
for i, n in enumerate(names):
    print(f"{i} ({n}): {counts[i]} objects")

Class mapping:
0 → Diode
1 → Transistor
2 → ceramic capacitors
3 → electrolytic capacitor

Train-only image counts:
0 (Diode): 14 objects
1 (Transistor): 12 objects
2 (ceramic capacitors): 18 objects
3 (electrolytic capacitor): 13 objects


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("derek-rxjzm").project("electronic-brown-oqh8h")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electronic-(Brown)-2 in yolov8:: 100%|██████████| 2493/2493 [00:00<00:00, 11156.02it/s]


In [ ]:
import yaml, os
from collections import defaultdict

# Load class names
yaml_path = os.path.join(dataset.location, "data.yaml")
with open(yaml_path, "r") as f:
    names = yaml.safe_load(f)["names"]

print("Class mapping:")
for i, n in enumerate(names):
    print(f"{i} → {n}")

# Count train-only images
counts = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if file.endswith(".txt"):
        with open(os.path.join(train_labels, file)) as f:
            for line in f:
                cls = int(line.split()[0])
                counts[cls] += 1

print("\nTrain-only image counts:")
for i, n in enumerate(names):
    print(f"{i} ({n}): {counts[i]} images")


Class mapping:
0 → POT
1 → RES

Train-only image counts:
0 (POT): 40 images
1 (RES): 3287 images


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("abc-qqksz").project("electronic-ezdmf")
version = project.version(23)
dataset = version.download("yolov8")
# only transistor and LED, delete everything else.

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Electronic-23 in yolov8:: 100%|██████████| 4864/4864 [00:00<00:00, 9280.33it/s] 


In [ ]:
import os, yaml

root = dataset.location
yaml_path = os.path.join(root, "data.yaml")

# ------------------------
# Load existing class names
# ------------------------
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

old_names = data["names"]
print("Old classes:", old_names)

# ------------------------
# The class to DELETE
# ------------------------
delete_class = "Led"    # <-- delete LED

# find its class ID
delete_id = None
for i, name in enumerate(old_names):
    if name.lower() == delete_class.lower():
        delete_id = i
        break

if delete_id is None:
    raise ValueError("LED class not found in data.yaml")

print("Deleting class:", delete_class, "with id:", delete_id)

# ------------------------
# Build new class list and ID remapping
# ------------------------
new_names = []
id_map = {}

new_id = 0
for old_id, cname in enumerate(old_names):
    if old_id == delete_id:
        continue  # skip LED entirely
    id_map[old_id] = new_id
    new_names.append(cname)
    new_id += 1

print("New class list:", new_names)
print("ID remapping:", id_map)

# ------------------------
# Rewrite all label files + delete LED images
# ------------------------
splits = ["train", "valid", "test"]
exts = [".jpg", ".jpeg", ".png"]

for split in splits:
    lbl_dir = os.path.join(root, split, "labels")
    img_dir = os.path.join(root, split, "images")

    if not os.path.exists(lbl_dir):
        continue

    for fname in os.listdir(lbl_dir):
        if not fname.endswith(".txt"):
            continue

        lbl_path = os.path.join(lbl_dir, fname)

        new_lines = []
        found_led = False

        with open(lbl_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                cls = int(parts[0])

                if cls == delete_id:
                    found_led = True
                    continue  # remove LED bbox

                # keep all other classes
                parts[0] = str(id_map[cls])
                new_lines.append(" ".join(parts))

        # delete image if file became empty
        if len(new_lines) == 0:
            os.remove(lbl_path)

            stem = fname[:-4]
            for ext in exts:
                img_file = os.path.join(img_dir, stem + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)
            continue

        # otherwise overwrite label file
        with open(lbl_path, "w") as f:
            f.write("\n".join(new_lines))

print("✔ Finished deleting LED class")

# ------------------------
# Update YAML file
# ------------------------
data["names"] = new_names
data["nc"] = len(new_names)

with open(yaml_path, "w") as f:
    yaml.dump(data, f)

print("✔ Updated data.yaml to", len(new_names), "classes")

Old classes: ['Diode', 'LED', 'Transistor', 'ceramic capacitors', 'electrolytic capacitor']
Deleting class: Led with id: 1
New class list: ['Diode', 'Transistor', 'ceramic capacitors', 'electrolytic capacitor']
ID remapping: {0: 0, 2: 1, 3: 2, 4: 3}
✔ Finished deleting LED class
✔ Updated data.yaml to 4 classes


In [ ]:
from collections import defaultdict

counts = defaultdict(int)
train_labels = os.path.join(dataset.location, "train/labels")

for file in os.listdir(train_labels):
    if not file.endswith(".txt"):
        continue

    with open(os.path.join(train_labels, file)) as f:
        for line in f:
            cls = int(line.split()[0])
            counts[cls] += 1

print("Train image counts:")
for i, name in enumerate(new_class_list):
    print(f"{i} ({name}): {counts[i]} images")

Train image counts:
0 (IC): 14 images
1 (Led): 12 images
2 (Transistor): 18 images
3 (dioda): 13 images


In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="rT5dIHK4PdVqtM72KTiY")
project = rf.workspace("universitas-negeri-semarang-ibchr").project("komponen-elektronika-skripsi")
version = project.version(11)
dataset = version.download("yolov8")
# only transistor, LED, diode, and IC; delete everything else.

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to komponen-elektronika-skripsi-11 in yolov8:: 100%|██████████| 5304/5304 [00:00<00:00, 9480.70it/s] 


In [ ]:
import yaml, os

yaml_path = os.path.join(dataset.location, "data.yaml")

print("Looking at:", yaml_path)

with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]

print("Class mapping:")
for i, name in enumerate(names):
    print(f"{i} → {name}")

Looking at: /content/komponen-elektronika-skripsi-11/data.yaml
Class mapping:
0 → IC
1 → Induktor
2 → Led
3 → Potensiometer
4 → Resistor
5 → Transistor
6 → dioda
7 → kapasitor-elco
8 → kapasitor-keramik
9 → sensor-ldr
10 → trimpot


In [ ]:
import os, yaml
from collections import defaultdict

root = dataset.location
yaml_path = os.path.join(root, "data.yaml")

# -----------------------------
# Load original YAML file
# -----------------------------
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

old_names = data["names"]
print("Original classes:")
for i, n in enumerate(old_names):
    print(f"{i}: {n}")

# -----------------------------
# CLASSES TO KEEP (LED REMOVED)
# -----------------------------
keep_classes = ["IC", "Transistor", "dioda"]   # <-- NO LED HERE

# match case-insensitive
keep_ids = {
    i: name for i, name in enumerate(old_names)
    if name.lower() in [k.lower() for k in keep_classes]
}

print("\nKeeping old IDs:", keep_ids)

# -----------------------------
# NEW INDEX MAPPING
# old → new
# -----------------------------
new_mapping = {}
new_class_list = []

for new_id, (old_id, cname) in enumerate(keep_ids.items()):
    new_mapping[old_id] = new_id
    new_class_list.append(cname)

print("\nNew mapping:", new_mapping)
print("New class list:", new_class_list)

# -----------------------------
# REWRITE LABEL FILES
# -----------------------------
splits = ["train", "valid", "test"]

for split in splits:
    labels_dir = os.path.join(root, split, "labels")
    images_dir = os.path.join(root, split, "images")

    if not os.path.exists(labels_dir):
        continue

    for fname in os.listdir(labels_dir):
        if not fname.endswith(".txt"):
            continue

        path = os.path.join(labels_dir, fname)

        new_lines = []
        with open(path, "r") as f:
            for line in f:
                parts = line.strip().split()
                old_id = int(parts[0])

                # keep only these three classes, drop LED
                if old_id in new_mapping:
                    parts[0] = str(new_mapping[old_id])
                    new_lines.append(" ".join(parts))

        # Delete label file AND image if empty after removing LED
        if len(new_lines) == 0:
            os.remove(path)
            base = fname[:-4]
            for ext in [".jpg", ".jpeg", ".png"]:
                img_file = os.path.join(images_dir, base + ext)
                if os.path.exists(img_file):
                    os.remove(img_file)
        else:
            with open(path, "w") as f:
                f.write("\n".join(new_lines))

print("\n✔ Finished rewriting label files. Only IC, Transistor, dioda remain (LED removed).")

# -----------------------------
# UPDATE YAML
# -----------------------------
data["names"] = new_class_list
data["nc"] = len(new_class_list)

with open(yaml_path, "w") as f:
    yaml.dump(data, f)

print("✔ Updated data.yaml to 3 classes:", new_class_list)

# -----------------------------
# COUNT TRAIN IMAGES
# -----------------------------
image_counts = defaultdict(int)

train_labels = os.path.join(root, "train/labels")

for file in os.listdir(train_labels):
    if not file.endswith(".txt"):
        continue

    classes_in_img = set()
    with open(os.path.join(train_labels, file)) as f:
        for line in f:
            cls = int(line.split()[0])
            classes_in_img.add(cls)

    for cls in classes_in_img:
        image_counts[cls] += 1

print("\nFinal TRAIN image counts (after LED removal):")
for i, name in enumerate(new_class_list):
    print(f"{i} ({name}): {image_counts[i]} images")

Original classes:
0: Diode
1: Transistor
2: ceramic capacitors
3: electrolytic capacitor

Keeping old IDs: {1: 'Transistor'}

New mapping: {1: 0}
New class list: ['Transistor']

✔ Finished rewriting label files. Only IC, Transistor, dioda remain (LED removed).
✔ Updated data.yaml to 3 classes: ['Transistor']

Final TRAIN image counts (after LED removal):
0 (Transistor): 12 images


In [ ]:
import os, cv2

root_images = "/content/Resistor-Value-Detector-2/train/images"
root_labels = "/content/Resistor-Value-Detector-2/train/labels"
RESISTOR_ID = 1  # RES class

# --- Rebuild resistor_images ---
resistor_images = set()

for lbl in os.listdir(root_labels):
    if lbl.endswith(".txt"):
        with open(os.path.join(root_labels, lbl)) as f:
            for line in f:
                if line.strip():
                    cid = int(line.split()[0])
                    if cid == RESISTOR_ID:
                        base = os.path.splitext(lbl)[0]
                        for ext in [".jpg", ".jpeg", ".png"]:
                            img_path = os.path.join(root_images, base + ext)
                            if os.path.exists(img_path):
                                resistor_images.add(base + ext)
                                break
                        break

print("Resistor images found:", len(resistor_images))

# --- Recompute sharpness ---
resistor_quality = []

for fname in resistor_images:
    img_path = os.path.join(root_images, fname)
    img = cv2.imread(img_path)
    if img is None:
        continue
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    resistor_quality.append((fname, fm))

print("Computed sharpness for:", len(resistor_quality))

# --- Sort sharpest → blurriest ---
resistor_quality_sorted = sorted(resistor_quality, key=lambda x: x[1], reverse=True)

print("Sorting complete.")


Resistor images found: 900
Computed sharpness for: 900
Sorting complete.


In [ ]:
KEEP_COUNT = 900

# sharpest → blurriest
resistor_quality_sorted = sorted(resistor_quality, key=lambda x: x[1], reverse=True)

keep_set = set([fname for fname, fm in resistor_quality_sorted[:KEEP_COUNT]])

print("Will keep:", len(keep_set), "images")

print("Sharpness threshold (last kept):", resistor_quality_sorted[KEEP_COUNT-1])


Will keep: 900 images
Sharpness threshold (last kept): ('5B-1R-T1-9_jpg.rf.cb83db38040144f5bd09df555e1a5a8c.jpg', np.float64(1010.1175311092375))


In [ ]:
import os

root_images = "/content/Resistor-Value-Detector-2/train/images"
root_labels = "/content/Resistor-Value-Detector-2/train/labels"

deleted_imgs = 0
deleted_labels = 0

for fname, fm in resistor_quality_sorted:
    if fname in keep_set:
        continue  # keep this one

    base = os.path.splitext(fname)[0]
    img_path = os.path.join(root_images, fname)
    lbl_path = os.path.join(root_labels, base + ".txt")

    if os.path.exists(img_path):
        os.remove(img_path)
        deleted_imgs += 1

    if os.path.exists(lbl_path):
        os.remove(lbl_path)
        deleted_labels += 1

print("Deleted images:", deleted_imgs)
print("Deleted labels:", deleted_labels)

Deleted images: 200
Deleted labels: 200


In [ ]:
import os

root_labels = "/content/Resistor-Value-Detector-2/train/labels"
root_images = "/content/Resistor-Value-Detector-2/train/images"

RESISTOR_ID = 1
remaining_resistor_images = set()

for lbl in os.listdir(root_labels):
    if not lbl.endswith(".txt"):
        continue

    with open(os.path.join(root_labels, lbl)) as f:
        for line in f:
            if line.strip():
                cid = int(line.split()[0])
                if cid == RESISTOR_ID:
                    base = os.path.splitext(lbl)[0]
                    # find corresponding image
                    for ext in [".jpg", ".jpeg", ".png"]:
                        img_path = os.path.join(root_images, base + ext)
                        if os.path.exists(img_path):
                            remaining_resistor_images.add(base + ext)
                            break
                    break

print("Remaining resistor images:", len(remaining_resistor_images))


Remaining resistor images: 900


In [ ]:
# removing LED from Brown-2
import os
from pathlib import Path
import yaml

# -------------------------------------------------------------
# 1. Target dataset folder (ONLY the one you want cleaned)
# -------------------------------------------------------------
target_ds = Path("/content/Electronic-(Brown)-2")

# -------------------------------------------------------------
# 2. Load YAML and find the LED class index
# -------------------------------------------------------------
yaml_path = target_ds / "data.yaml"
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]

# Normalize names to lowercase for finding LED
led_names = ["led", "leds", "led_3mm", "led_5mm", "3mm_led", "5mm_led"]
led_idx = None

for i, n in enumerate(names):
    if n.lower() in led_names:
        led_idx = i
        break

if led_idx is None:
    raise ValueError("❌ LED class not found in data.yaml names list!")

print(f"✔ LED local class index = {led_idx}")
print()

# -------------------------------------------------------------
# 3. Delete LED images + labels inside this dataset only
# -------------------------------------------------------------
splits = ["train", "valid", "test"]
deleted_count = 0

for split in splits:
    labels_dir = target_ds / split / "labels"
    images_dir = target_ds / split / "images"

    if not labels_dir.exists():
        continue

    for txt_path in labels_dir.glob("*.txt"):
        lines = txt_path.read_text().splitlines()

        has_led = any(
            line.strip() and int(line.split()[0]) == led_idx
            for line in lines
        )

        if has_led:
            img_path = images_dir / (txt_path.stem + ".jpg")
            if not img_path.exists():
                # try png
                img_path = images_dir / (txt_path.stem + ".png")

            # Delete label
            os.remove(txt_path)

            # Delete image if exists
            if img_path.exists():
                os.remove(img_path)

            deleted_count += 1

print(f"\n✅ Deleted {deleted_count} LED images+labels from Electronic-(Brown)-2")

✔ LED local class index = 0


✅ Deleted 0 LED images+labels from Electronic-(Brown)-2


In [ ]:
import os, shutil, yaml
from pathlib import Path

# ===================== 1. Where your datasets are =====================
ROOT = Path("/content")  # change if needed

# TODO: put ONLY the folders that still exist and you want to merge
dataset_dirs = [
    "Electric-Components-1",
    "electronic-components-3",
    "Electronic-components-detection-4",
    "Electronics-Component-7",
    "Electronic-components-recognition-1",
    "Electronic-(Brown)-2",
    "Electronic-23",
    "Resistor-Value-Detector-2",
    "komponen-elektronika-skripsi-11",
]

dataset_dirs = [ROOT / d for d in dataset_dirs]

merged_root = ROOT / "merged_dataset"
splits = ["train", "valid", "test"]

# ===================== 2. Class-name normalization =====================
# EVERYTHING here is case-insensitive. Keys are lowercase.
normalize_name = {
    # Resistor / potentiometer
    "res": "resistor",
    "resistor": "resistor",
    "power-resistor": "resistor",
    "pot": "potentiometer",
    "potentiometer": "potentiometer",
    "potenciometer": "potentiometer",
    "potentiometer_": "potentiometer",
    "preset": "potentiometer",

    # Capacitors - ceramic
    "ceramic-capacitor": "ceramic_capacitor",
    "ceramic capacitor": "ceramic_capacitor",
    "ceramic_capacitor": "ceramic_capacitor",
    "capacitor-ceramico": "ceramic_capacitor",
    "ceramic capacitors": "ceramic_capacitor",

    # Capacitors - electrolytic
    "electrolytic-capacitor": "electrolytic_capacitor",
    "electrolytic capacitor": "electrolytic_capacitor",
    "electrolytic_capacitor": "electrolytic_capacitor",
    "capacitor-electrolitico": "electrolytic_capacitor",

    # Capacitors - polyester
    "polyester-capacitor": "polyester_capacitor",
    "capacitor-poliester": "polyester_capacitor",

    # LED variants
    "led": "led",
    "led_3mm": "led",
    "3mm_led": "led",
    "led_5mm": "led",
    "5mm_led": "led",
    "leds": "led",
    "led (5mm)": "led",

    # Diode variants
    "diode": "diode",
    "diodo": "diode",
    "dioda": "diode",
    "zener_diode": "diode",

    # Transistor variants
    "transistor": "transistor",
    "to-92": "transistor",
    "to-220": "transistor",

    # IC
    "ic": "ic",
    "integrated_circuit": "ic",

    # If you see weird names in the printout, add them here ↓
}

def norm(name: str) -> str:
    key = name.strip()
    lk = key.lower()
    return normalize_name.get(lk, key)  # default: keep original name


# ===================== 3. Read & normalize class names =====================
all_names_lists = []
all_norm_names_lists = []

for ds in dataset_dirs:
    data_yaml_path = ds / "data.yaml"
    if not data_yaml_path.exists():
        raise FileNotFoundError(f"{data_yaml_path} not found. Check folder: {ds}")

    with open(data_yaml_path, "r") as f:
        data = yaml.safe_load(f)

    names = data.get("names")
    # names can be list or dict {0: 'x', 1: 'y'}
    if isinstance(names, dict):
        names = [names[k] for k in sorted(names.keys(), key=lambda x: int(x))]

    norm_names = [norm(n) for n in names]

    print(f"\nDataset: {ds.name}")
    print("  Original names:", names)
    print("  Normalized names:", norm_names)

    all_names_lists.append(names)
    all_norm_names_lists.append(norm_names)

# ===================== 4. Build global class list =====================
global_names = []
for names in all_norm_names_lists:
    for n in names:
        if n not in global_names:
            global_names.append(n)

print("\nGlobal classes (after normalization):")
for i, n in enumerate(global_names):
    print(f"{i}: {n}")

# ===================== 5. Local → global index maps =====================
local_to_global_maps = []
for norm_names in all_norm_names_lists:
    mapping = {}
    for local_idx, name in enumerate(norm_names):
        global_idx = global_names.index(name)
        mapping[local_idx] = global_idx
    local_to_global_maps.append(mapping)

print("\nLocal→Global maps:")
for ds, mapping in zip(dataset_dirs, local_to_global_maps):
    print(f"{ds.name}: {mapping}")

# ===================== 6. Create fresh merged structure =====================
if merged_root.exists():
    shutil.rmtree(merged_root)

for split in splits:
    (merged_root / split / "images").mkdir(parents=True, exist_ok=True)
    (merged_root / split / "labels").mkdir(parents=True, exist_ok=True)

# ===================== 7. Copy images + remap labels =====================
for ds_idx, ds in enumerate(dataset_dirs):
    mapping = local_to_global_maps[ds_idx]

    for split in splits:
        images_dir = ds / split / "images"
        labels_dir = ds / split / "labels"

        if not images_dir.exists():
            print(f"[{ds.name}] No '{split}' split, skipping.")
            continue

        for img_path in images_dir.glob("*"):
            if not img_path.is_file():
                continue

            # Give each image a unique prefix from its dataset index
            new_img_name = f"{ds_idx}_{img_path.name}"
            new_img_path = merged_root / split / "images" / new_img_name

            label_path = labels_dir / (img_path.stem + ".txt")
            new_label_name = f"{ds_idx}_{img_path.stem}.txt"
            new_label_path = merged_root / split / "labels" / new_label_name

            shutil.copy(img_path, new_img_path)

            if label_path.exists():
                lines = label_path.read_text().strip().splitlines()
                new_lines = []
                for line in lines:
                    if not line.strip():
                        continue
                    parts = line.split()
                    cls_local = int(parts[0])
                    if cls_local not in mapping:
                        raise ValueError(
                            f"Unexpected class id {cls_local} in {label_path} from dataset {ds.name}"
                        )
                    cls_global = mapping[cls_local]
                    parts[0] = str(cls_global)
                    new_lines.append(" ".join(parts))
                new_label_path.write_text("\n".join(new_lines))
            else:
                # If some dataset has images without labels
                print(f"⚠️ No label for image {img_path}")

print("\n✅ Finished merging into", merged_root)

# ===================== 8. Write merged data.yaml =====================
merged_yaml = {
    "path": str(merged_root),
    "train": "train/images",
    "val": "valid/images",
    "test": "test/images",  # okay if empty
    "names": global_names,
    "nc": len(global_names),
}

with open(merged_root / "data.yaml", "w") as f:
    yaml.safe_dump(merged_yaml, f)

print("✅ merged data.yaml written with", len(global_names), "classes.")


Dataset: Electric-Components-1
  Original names: ['ceramic_capacitor', 'potentiometer', 'resistor']
  Normalized names: ['ceramic_capacitor', 'potentiometer', 'resistor']

Dataset: electronic-components-3
  Original names: ['ceramic-capacitor', 'diode', 'electrolytic-capacitor', 'polyester-capacitor', 'resistor', 'transistor']
  Normalized names: ['ceramic_capacitor', 'diode', 'electrolytic_capacitor', 'polyester_capacitor', 'resistor', 'transistor']

Dataset: Electronic-components-detection-4
  Original names: ['capacitor-ceramico', 'capacitor-electrolitico', 'capacitor-poliester', 'diodo', 'ic', 'led']
  Normalized names: ['ceramic_capacitor', 'electrolytic_capacitor', 'polyester_capacitor', 'diode', 'ic', 'led']

Dataset: Electronics-Component-7
  Original names: ['Transistor']
  Normalized names: ['transistor']

Dataset: Electronic-components-recognition-1
  Original names: ['LED', 'electrolytic capacitor', 'ceramic capacitor', 'potentiometer', 'resistor', 'diode']
  Normalized na

In [ ]:
# deleting unlabeled images
from pathlib import Path

merged_root = Path("/content/merged_dataset")   # <-- your merged folder
splits = ["train", "valid", "test"]

for split in splits:
    imgs_dir = merged_root / split / "images"
    labels_dir = merged_root / split / "labels"

    if not imgs_dir.exists():
        print(f"{split}: no images folder, skipping")
        continue

    removed = 0
    total = 0

    for img_path in imgs_dir.iterdir():
        if not img_path.is_file():
            continue

        total += 1
        stem = img_path.stem
        label_path = labels_dir / f"{stem}.txt"

        # DELETE image if no corresponding label exists
        if not label_path.exists():
            img_path.unlink()
            removed += 1

    print(f"\n--- {split.upper()} ---")
    print(f"Total images checked: {total}")
    print(f"Unlabeled images removed: {removed}")



--- TRAIN ---
Total images checked: 6549
Unlabeled images removed: 1952

--- VALID ---
Total images checked: 1430
Unlabeled images removed: 360

--- TEST ---
Total images checked: 534
Unlabeled images removed: 171


In [ ]:
from pathlib import Path
import yaml
from collections import defaultdict

root = Path("/content/merged_dataset")   # <-- YOUR MERGED FOLDER
yaml_path = root / "data.yaml"

# -----------------------
# Load class names
# -----------------------
with open(yaml_path, "r") as f:
    data = yaml.safe_load(f)

names = data["names"]
if isinstance(names, dict):  # handle dict-form YAML
    names = [names[i] for i in sorted(names.keys(), key=int)]

nc = len(names)

print("Classes:", names)

# -----------------------
# Count images per class
# -----------------------
splits = ["train", "valid", "test"]
results = {split: [0]*nc for split in splits}

for split in splits:
    lbl_dir = root / split / "labels"
    if not lbl_dir.exists():
        continue

    for lbl_file in lbl_dir.glob("*.txt"):
        # find which classes appear in the image
        classes_in_img = set()

        with open(lbl_file, "r") as f:
            for line in f:
                cls = int(line.split()[0])
                classes_in_img.add(cls)

        # add +1 image count to each class that appears
        for cls in classes_in_img:
            results[split][cls] += 1

# -----------------------
# PRINT TABLE
# -----------------------
print("\nClass-wise image counts in merged_dataset:\n")
print(f"{'id':<4}{'name':<28}{'train':<8}{'valid':<8}{'test':<8}{'total'}")

for i, name in enumerate(names):
    tr = results["train"][i]
    va = results["valid"][i]
    te = results["test"][i]
    total = tr + va + te
    print(f"{i:<4}{name:<28}{tr:<8}{va:<8}{te:<8}{total}")

Classes: ['ceramic_capacitor', 'potentiometer', 'resistor', 'diode', 'electrolytic_capacitor', 'polyester_capacitor', 'transistor', 'ic', 'led']

Class-wise image counts in merged_dataset:

id  name                        train   valid   test    total
0   ceramic_capacitor           244     58      21      323
1   potentiometer               163     47      19      229
2   resistor                    968     540     138     1646
3   diode                       322     60      24      406
4   electrolytic_capacitor      379     52      27      458
5   polyester_capacitor         79      9       3       91
6   transistor                  1042    156     67      1265
7   ic                          731     13      7       751
8   led                         898     173     79      1150


In [ ]:
from collections import defaultdict
import yaml

led_idx = global_names.index("led")

led_images_by_dataset = defaultdict(int)

for ds_idx, ds in enumerate(dataset_dirs):
    mapping = local_to_global_maps[ds_idx]

    for split in splits:
        labels_dir = ds / split / "labels"
        if not labels_dir.exists():
            continue

        for txt in labels_dir.glob("*.txt"):
            has_led = False
            for line in txt.read_text().splitlines():
                line = line.strip()
                if not line:
                    continue
                parts = line.split()
                cls_local = int(parts[0])
                cls_global = mapping[cls_local]
                if cls_global == led_idx:
                    has_led = True
                    break

            if has_led:
                led_images_by_dataset[ds.name] += 1

print("LED images contributed by each dataset:")
for name, cnt in led_images_by_dataset.items():
    print(f"{name}: {cnt}")
print("Total LED images across all sources:", sum(led_images_by_dataset.values()))

LED images contributed by each dataset:
Electronic-components-detection-4: 131
Electronic-components-recognition-1: 307
Electronic-(Brown)-2: 1241
Electronic-23: 502
komponen-elektronika-skripsi-11: 210
Total LED images across all sources: 2391


In [ ]:
import os, shutil, glob

merged_path = "/content/merged_dataset"

# ----- Delete merged dataset -----
if os.path.exists(merged_path):
    shutil.rmtree(merged_path)
    print("Deleted:", merged_path)
else:
    print("No merged_dataset found.")

# ----- Delete ALL YOLO caches anywhere in /content -----
cache_paths = glob.glob("/content/**/cache*", recursive=True)

for c in cache_paths:
    if os.path.isdir(c):
        shutil.rmtree(c)
        print("Deleted cache folder:", c)
    elif os.path.isfile(c):
        os.remove(c)
        print("Deleted cache file:", c)

print("\n✔ All caches cleaned. You can safely re-merge now.")

Deleted: /content/merged_dataset

✔ All caches cleaned. You can safely re-merge now.


In [ ]:
from pathlib import Path
import shutil

merged_root = Path("/content/merged_components")  # adjust if different

if merged_root.exists():
    shutil.rmtree(merged_root)
    print("Deleted existing:", merged_root)
else:
    print("No merged_components folder found at:", merged_root)

Deleted existing: /content/merged_components


In [ ]:
!zip -r /content/merged_dataset.zip /content/merged_dataset

	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (/content/merged_dataset.zip)


In [ ]:
from google.colab import files
files.download('/content/merged_dataset.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -rf electronic-components-3
!rm -rf Resistor-Value-Detector-2
!rm -rf Electric-Components-1


In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
files.download('runs/detect/train2/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!unzip -o /content/merged_dataset.zip "content/merged_dataset/*" -d /content/

Archive:  /content/merged_dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/merged_dataset.zip or
        /content/merged_dataset.zip.zip, and cannot find /content/merged_dataset.zip.ZIP, period.


In [ ]:
!mv /content/content/merged_dataset /content/merged_dataset

mv: cannot stat '/content/content/merged_dataset': No such file or directory


In [ ]:
import yaml
from pathlib import Path

root = Path("/content/merged_dataset")

# Load classes
with open(root / "data.yaml", "r") as f:
    data = yaml.safe_load(f)
names = data["names"]
nc = len(names)

# Storage for counts
counts = {i: 0 for i in range(nc)}

splits = ["train", "valid", "test"]

for split in splits:
    labels_dir = root / split / "labels"
    if not labels_dir.exists():
        continue

    for txt in labels_dir.glob("*.txt"):
        found_classes = set()
        for line in txt.read_text().splitlines():
            if not line.strip():
                continue
            cls = int(line.split()[0])
            found_classes.add(cls)

        # Count each class once per image
        for c in found_classes:
            counts[c] += 1

# Print results
print("\nClass-wise image counts:")
for i, name in enumerate(names):
    print(f"{i:<3} {name:<25} {counts[i]}")


Class-wise image counts:
0   ceramic_capacitor         323
1   potentiometer             229
2   resistor                  1646
3   diode                     406
4   electrolytic_capacitor    458
5   polyester_capacitor       91
6   transistor                1265
7   ic                        751
8   led                       1150


In [ ]:
from ultralytics import YOLO

model = YOLO("yolov8s.pt")  # start with s

model.train(
    data="/content/merged_dataset/data.yaml",
    epochs=100,
    imgsz=640,
    batch=64,        # lower to 32 if you ever hit CUDA OOM
    device=0,        # A100
    workers=8,
    patience=25,
    cos_lr=True,
    optimizer="SGD", # fine; you can also omit this and let YOLO choose
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, i

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ad3d0dd7a10>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
from google.colab import files
files.download("runs/detect/train/weights/best.pt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO

model = YOLO("runs/detect/train/weights/best.pt")  # or train2, etc.
model.val(data="/content/merged_dataset/data.yaml", split="test", plots=True)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
Model summary (fused): 72 layers, 11,129,067 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 654.4±228.2 MB/s, size: 24.8 KB)
val: Scanning /content/merged_dataset/test/labels... 363 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 363/363 1.5Kit/s 0.2s
val: New cache created: /content/merged_dataset/test/labels.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 53, len(boxes) = 832. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 23/23 7.3it/s 3.1s
                   all        363        832      0.715      0.756      0.749      0.483
     ceramic_capacitor         21         47      0.762

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ad3ce1a4b90>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
!ls /content/runs/detect/train


args.yaml			 results.csv	      val_batch0_labels.jpg
BoxF1_curve.png			 results.png	      val_batch0_pred.jpg
BoxP_curve.png			 train_batch0.jpg     val_batch1_labels.jpg
BoxPR_curve.png			 train_batch1.jpg     val_batch1_pred.jpg
BoxR_curve.png			 train_batch2.jpg     val_batch2_labels.jpg
confusion_matrix_normalized.png  train_batch6480.jpg  val_batch2_pred.jpg
confusion_matrix.png		 train_batch6481.jpg  weights
labels.jpg			 train_batch6482.jpg


In [ ]:
from ultralytics import YOLO

# load your already-trained model
model = YOLO("runs/detect/train/weights/best.pt")

# fine-tune with higher resolution
model.train(
    data="/content/merged_dataset/data.yaml",
    epochs=40,      # short fine-tune, not full retrain
    imgsz=800,      # higher resolution
    batch=32,       # safer batch for 800 on A100
    device=0,
    workers=8,
    patience=20,
    cos_lr=True,
)

Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (NVIDIA A100-SXM4-80GB, 81222MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/content/merged_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=40, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=800, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=runs/detect/train/weights/best.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5, 6, 7, 8])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ad3d811a210>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.0470

In [ ]:
!ls -lt runs/detect

total 12
drwxr-xr-x 3 root root 4096 Dec  6 04:52 train2
drwxr-xr-x 2 root root 4096 Dec  6 04:15 val
drwxr-xr-x 3 root root 4096 Dec  6 03:43 train
